In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
#loading the dataset to pandas dataframe
original_news_dataset=pd.read_csv('/content/FakeNewsNet.csv')

In [5]:
original_news_dataset.shape

(23196, 5)

In [6]:
#print first five rows and coloums of dataframe
original_news_dataset.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [7]:
 #counting the number of missing value in the dataset
original_news_dataset.isnull().sum()

,0
title,0
news_url,330
source_domain,330
tweet_num,0
real,0


In [8]:
#Replace the null value with the empty string
original_news_dataset = original_news_dataset.fillna('')

In [9]:
X = original_news_dataset[['title']]

In [10]:
Y = original_news_dataset['real']
print(X)
print(Y)

                                                   title
0      Kandi Burruss Explodes Over Rape Accusation on...
1      People's Choice Awards 2018: The best red carp...
2      Sophia Bush Sends Sweet Birthday Message to 'O...
3      Colombian singer Maluma sparks rumours of inap...
4      Gossip Girl 10 Years Later: How Upper East Sid...
...                                                  ...
23191  Pippa Middleton wedding: In case you missed it...
23192  Zayn Malik & Gigi Hadid’s Shocking Split: Why ...
23193  Jessica Chastain Recalls the Moment Her Mother...
23194  Tristan Thompson Feels "Dumped" After Khloé Ka...
23195  Kelly Clarkson Performs a Medley of Kendrick L...

[23196 rows x 1 columns]
0        1
1        1
2        1
3        1
4        1
        ..
23191    1
23192    0
23193    1
23194    0
23195    1
Name: real, Length: 23196, dtype: int64


In [12]:
port_stem = PorterStemmer()

In [18]:
def stemming(content):
  # Assign an initial value to stemmed_content
  steammed_content = re.sub('[^a-zA-Z]',' ',content)   # re.sub('[^a-zA-Z]  remove content that not present in alphabetic set that is , !,@,etc
  stemmed_content = steammed_content.lower()            # conver alphabet converted case
  stemmed_content = stemmed_content.split()
  # Correct stopwords.words to stopwords.words
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content) # join with space to form sentence
  return stemmed_content

In [20]:
original_news_dataset['content'] = original_news_dataset['title'].apply(stemming) # Replace '' with the actual column name you want to process, for instance, 'title' if that's where the text content resides.

In [21]:
print(original_news_dataset['content'])

0        kandi burruss explod rape accus real housew at...
1                   peopl choic award best red carpet look
2        sophia bush send sweet birthday messag one tre...
3        colombian singer maluma spark rumour inappropr...
4        gossip girl year later upper east sider shock ...
                               ...                        
23191    pippa middleton wed case miss pippa marri lace...
23192    zayn malik gigi hadid shock split chanc reunit...
23193    jessica chastain recal moment mother boyfriend...
23194    tristan thompson feel dump khlo kardashian ref...
23195    kelli clarkson perform medley kendrick lamar h...
Name: content, Length: 23196, dtype: object


In [22]:
# seprating the data and label
X=original_news_dataset['content'].values
Y=original_news_dataset['real'].values

In [23]:
print(X)

['kandi burruss explod rape accus real housew atlanta reunion video'
 'peopl choic award best red carpet look'
 'sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva'
 ... 'jessica chastain recal moment mother boyfriend slap kick genit'
 'tristan thompson feel dump khlo kardashian refus let move la home exclus'
 'kelli clarkson perform medley kendrick lamar humbl hit billboard music award']


In [24]:
print(Y)

[1 1 1 ... 1 0 1]


In [25]:
Y.shape

(23196,)

In [26]:
# converting the textual to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X=vectorizer.transform(X)

In [27]:
print(X)

  (0, 53)	0.2721149519149911
  (0, 576)	0.32205761515065545
  (0, 1524)	0.4060028523437122
  (0, 3739)	0.37202135470266046
  (0, 5246)	0.27372476382132127
  (0, 5895)	0.4060028523437122
  (0, 8975)	0.3165911519884338
  (0, 9020)	0.24249280188561428
  (0, 9287)	0.26647854102470714
  (0, 11959)	0.2217815349642914
  (1, 643)	0.31690546637483147
  (1, 986)	0.34762560771735407
  (1, 1710)	0.3978429114527965
  (1, 1975)	0.43378159796649224
  (1, 6573)	0.3420111696667716
  (1, 8315)	0.4067948402953306
  (1, 9079)	0.387076508000102
  (2, 1069)	0.19438975915734968
  (2, 1361)	0.37698546923989756
  (2, 1528)	0.3313932058546551
  (2, 1529)	0.2679606830015344
  (2, 2132)	0.22767646060985314
  (2, 3657)	0.25142880383632443
  (2, 5100)	0.22534648956135603
  (2, 5104)	0.2418360754414745
  :	:
  (23193, 9046)	0.32490265953129527
  (23193, 10239)	0.39707689058614987
  (23194, 3306)	0.3321442681266319
  (23194, 3706)	0.2587855648260621
  (23194, 3907)	0.2822937698497612
  (23194, 5165)	0.276602517531464

Spltting the dataset to traning and test data

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

Traning the model:Logistic Regression

In [31]:
model = LogisticRegression()

In [33]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

In [35]:
# accuracy score for traning data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [36]:
print('Accuracy score of the traning data:', training_data_accuracy)

Accuracy score of the traning data: 0.8600991593015737


In [37]:
# accuracy score for test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [38]:
print('Accuracy score of the test data:', test_data_accuracy)

Accuracy score of the test data: 0.8349137931034483


Making a predictive system

In [50]:
X_new = X_test[2]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [51]:
print(Y_test[2])

1
